In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

: 

In [3]:
df = pd.read_csv('/tempo/processed-files/df.csv')
embedding = np.load('/tempo/processed-files/embeddings.npy') 
new_labels = np.load('/tempo/processed-files/new_labels.npy', allow_pickle=True)

In [9]:
df['label'] = new_labels

In [52]:
# labels that share the same relationship group
unified_conv_labels = {
    0:0, 2:2, 3:2, 4:2, 5:2, 6:2, 7:7, 8:7, 9:7, 10:10, 11:10, 12:10, 13:10, 15:0, 17:17, 19:17, 20:20, 21:21, 22:21, 23:23, 24:23, 25:23, 27:27, 28:27, 29:27, 30:27, 31:27
    }

In [11]:
# Get 20% of the dataset for training
df_train = df.sample(frac=0.2, random_state=0)

# unify the labels
# df_train['label'] = df_train['Relation'].map(unified_conv_labels)
df_train.drop(columns = ['Relation'], inplace=True)
df_train['label'] = df_train['label'].astype('category')
df_train['label'] = df_train['label'].cat.codes

In [16]:
df_train

,Word A,Word B,Sentence,label
1302495,contrast,display,Display technology must offer high dynamic ran...,18
411558,neural network,neural network model,"Third, neural network models are easier to ove...",18
764933,ab,simulation,We use our simulations to evaluate the torsion...,17
1342397,knowledge model,ontology,Graph knowledge models and ontologies are very...,0
828861,parameter,resolution,The haloes are extracted from the OverWhelming...,0
...,...,...,...,...
480383,clustering,markov chain,"With the present observational data of CMB, Ty...",17
902118,cnn,transfer learning,Experimental results indicate that transfer le...,18
299923,objective function,parameter,A penalty on the transformation parameters is ...,0
911761,annealing,simulated annealing,"In a finite trap, we argue using an LDA and si...",17


In [17]:
# unique labels
output = len(df_train['label'].unique())

In [22]:
# Make a sequential model for multi-class classification of the unified labels
# The input of the model is the sentence embedding of size 384. The output is the label. The last layer is a softmax layer into one of 27 relations. The loss function is categorical crossentropy.

embedding_model = keras.Sequential([
        layers.Dense(300, activation='relu', input_shape=(384,)),
    ])


model = keras.Sequential([
    embedding_model,
    layers.Dense(output, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# create a callback in case the model stops improving
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model.fit(
    embedding[df_train.index],
    df_train['label'].to_numpy(),
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[callback]
)


Epoch 1/50
7202/7202 [==============================] - 11s 1ms/step - loss: 1.1330 - accuracy: 0.5971 - val_loss: 1.0789 - val_accuracy: 0.6002
Epoch 2/50
7202/7202 [==============================] - 10s 1ms/step - loss: 1.0195 - accuracy: 0.6262 - val_loss: 1.0374 - val_accuracy: 0.6217
Epoch 3/50
7202/7202 [==============================] - 10s 1ms/step - loss: 0.9721 - accuracy: 0.6393 - val_loss: 1.0315 - val_accuracy: 0.6209
Epoch 4/50
7202/7202 [==============================] - 10s 1ms/step - loss: 0.9382 - accuracy: 0.6493 - val_loss: 1.0242 - val_accuracy: 0.6268
Epoch 5/50
7202/7202 [==============================] - 10s 1ms/step - loss: 0.9109 - accuracy: 0.6569 - val_loss: 1.0321 - val_accuracy: 0.6257
Epoch 6/50
7202/7202 [==============================] - 10s 1ms/step - loss: 0.8879 - accuracy: 0.6650 - val_loss: 1.0287 - val_accuracy: 0.6280
Epoch 7/50
7202/7202 [==============================] - 10s 1ms/step - loss: 0.8681 - accuracy: 0.6707 - val_loss: 1.0418 - val_ac

In [23]:
# Use the embedding model to get the embeddings of the whole dataset
separated_embeddings = embedding_model.predict(embedding)

  159/45008 [..............................] - ETA: 28s 

2022-08-14 14:14:57.440970: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2212208640 exceeds 10% of free system memory.


45008/45008 [==============================] - 31s 695us/step


2022-08-14 14:15:38.127907: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1728288000 exceeds 10% of free system memory.


In [26]:
# Save the embeddings
np.save('/tempo/processed-files/sep_embeddings.npy', separated_embeddings)